# TensorFlow Semi-supervised Object Detection Architecture (TSODA)
Welcome to this project, I'll explain to you the necessary steps to run this application and train your own semi-supervised model.

If you forgot something, here is the original tutorial: https://medium.com/p/757b9c88f270/edit.

Also check the GitHub repository: https://github.com/AlvaroCavalcante/tf-models.



# Initial configurations

Here you can find more models if you don't want to use SSD: [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), and here the respective configuration files: [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
repo_url = 'https://github.com/AlvaroCavalcante/tf-models' # replace by your repo

MODEL = 'ssd_inception_v2_coco_2018_01_28' # replace by the model you want use

pipeline_file = 'ssd_inception_v2_coco.config'

# Model hyperparameters
num_steps = 2500
num_eval_steps = 50
batch_size = 16

## Clone your repository which has your images and scripts!



In [ ]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Cloning into 'tf-models'...
remote: Enumerating objects: 40606, done.
remote: Total 40606 (delta 0), reused 0 (delta 0), pack-reused 40606
Receiving objects: 100% (40606/40606), 570.92 MiB | 34.11 MiB/s, done.
Resolving deltas: 100% (26595/26595), done.
/content/tf-models
Already up to date.


## Installing the requirements


In [ ]:
%cd /content

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install gast==0.3.3

!pip install tf_slim

!pip install virtualenv

%cd /content/tf-models/research
!protoc object_detection/protos/*.proto --python_out=.

import shutil
import os
os.environ['PYTHONPATH'] += ':/content/tf-models/research/:/content/tf-models/research/slim/'

!python object_detection/builders/model_builder_test.py
!pip install tensorflow-object-detection-api

/content
Selecting previously unselected package python-bs4.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

# Creating the necessary methods
These methods will be used in the iteration of the semi-supervised model.


## Preparing the **tfrecord** files.

The **generate_tfrecord()** method is responsable to convert the JPG + XML files to CSV and then to TF record, but befone is checked if the record files already exists and delete the files if condition is True.

This is necessary because this method is called in each iteration, generating a new version of TF record with the labeled images, avoiding any possible error.


In [ ]:
def delete_files():
  path = '/content/tf-models/research/object_detection/'
  files = ['train.record', 'test.record', 'train_labels.csv', 'test_labels.csv']
  
  for current_file in files:
    os.remove(path + current_file)

  print('Deleted files')

In [ ]:
def generate_tfrecord(repo_dir_path):
  %cd {repo_dir_path}
  
  if os.path.isfile('/content/tf-models/research/object_detection/train.record'):
    delete_files()
  
  !python research/object_detection/xml_to_csv.py -i research/object_detection/train_images -o research/object_detection/train_labels.csv -l research/object_detection/label_map

  !python research/object_detection/xml_to_csv.py -i research/object_detection/test_images -o research/object_detection/test_labels.csv

  !python research/object_detection/generate_tfrecord.py --csv_input=research/object_detection/train_labels.csv --output_path=research/object_detection/train.record --img_path=research/object_detection/train_images --label_map research/object_detection/label_map/label_map.pbtxt

  !python research/object_detection/generate_tfrecord.py --csv_input=research/object_detection/test_labels.csv --output_path=research/object_detection/test.record --img_path=research/object_detection/test_images --label_map research/object_detection/label_map/label_map.pbtxt

Call the method first time and define the paths

In [ ]:
generate_tfrecord(repo_dir_path)

test_record_fname = '/content/tf-models/research/object_detection/test.record'
train_record_fname = '/content/tf-models/research/object_detection/train.record'
label_map_pbtxt_fname = '/content/tf-models/research/object_detection/label_map/label_map.pbtxt'

tf_log_path = '/content/tf-models/research/object_detection/events.out.tfevents.1576020673.0083b462c1a8'

/content/tf-models
Successfully converted xml to csv.
Generate `research/object_detection/label_map/label_map.pbtxt`
Successfully converted xml to csv.
2020-07-21 00:39:11.464688: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/tf-models/research/object_detection/train.record
2020-07-21 00:39:19.720019: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/tf-models/research/object_detection/test.record


## Download base model for transfer learning
Transfer learning is used based on a pre-trained model in COCO dataset!


In [ ]:
%cd /content/tf-models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/tf-models/research/pretrained_model' #saved here

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/tf-models/research



## Configuring the training pipeline
The method **change_pipeline()** will be called to update the parameters of the model configuration file. In practice, it's necessary to update the total number of epochs and the checkpoint path, once just in the first iteration we'll use the downloaded model, as in the next ones our own treined model checkpoint will be updated during training.  


In [ ]:
import os

pipeline_fname = os.path.join('/content/tf-models/research/object_detection/samples/configs/', pipeline_file)
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt") 

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    print(categories)
    category_index = label_map_util.create_category_index(categories)
    print(category_index)
    return len(category_index.keys())

In [ ]:
def change_pipeline():
  import re

  num_classes = get_num_classes(label_map_pbtxt_fname) 

  with open(pipeline_fname) as f:
      s = f.read()
  with open(pipeline_fname, 'w') as f:
      
      # fine_tune_checkpoint
      s = re.sub('fine_tune_checkpoint: ".*?"',
                'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
      
      # tfrecord files train and test.
      s = re.sub(
          '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
      s = re.sub(
          '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

      # label_map_path
      s = re.sub(
          'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

      # Set training batch_size.
      s = re.sub('batch_size: [0-9]+',
                'batch_size: {}'.format(batch_size), s)

      # Set training steps, num_steps
      s = re.sub('num_steps: [0-9]+',
                'num_steps: {}'.format(num_steps), s)
      
      # Set number of classes num_classes.
      s = re.sub('num_classes: [0-9]+',
                'num_classes: {}'.format(num_classes), s)
      f.write(s)

In [ ]:
change_pipeline()
!cat {pipeline_fname} # this is your final result pipeline config

[{'id': 1, 'name': 'cat'}, {'id': 2, 'name': 'dog'}]
{1: {'id': 1, 'name': 'cat'}, 2: {'id': 2, 'name': 'dog'}}
# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 2
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
    

## Restarting the training directory
Every iteration we remove the model dir to avoid erros, the checkpoint is restored to continue the training.

In [ ]:
def start_new_training_dir():
  model_dir = 'training/'
  !rm -rf {model_dir}
  os.makedirs(model_dir, exist_ok=True)

model_dir = 'training/'
start_new_training_dir()

# Training process
The following steps are the most important for the semi supervised iteration.

## Exporting model
The **export_model()** method is called just after the training finishes, saving a checkpoint and the saved_model.pb which is used to infer new images.

A improvement that could be done is instead of use the saved_model.pb try to load the checkpoints and use them to infer the images, creating a even faster iteration.

**Note:** We are accepting collaborators for the project :)

In [ ]:
def export_model():
  import re
  import numpy as np

  output_directory = './fine_tuned_model'

  lst = os.listdir(model_dir)
  lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
  steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
  last_model = lst[steps.argmax()].replace('.meta', '')

  last_model_path = os.path.join(model_dir, last_model)
  print(last_model_path)
  !source tf1/bin/activate; python /content/tf-models/research/object_detection/export_inference_graph.py \
      --input_type=image_tensor \
      --pipeline_config_path={pipeline_fname} \
      --output_directory={output_directory} \
      --trained_checkpoint_prefix={last_model_path}

## Training the model
To train the model we use the **train_model()** method, that just run the training script with the previous specified parameters.

The beginning of the command is the virtual env initialization, which is explained below.

In [ ]:
def train_model():
  !source tf1/bin/activate; python /content/tf-models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir={model_dir} \
      --alsologtostderr \
      --num_train_steps={num_steps} \
      --num_eval_steps={num_eval_steps}

## Inferences!
The model inference is done in the following method, which just enter in the research folder and run the inference_from_model.py 

In [ ]:
def inference_from_model():
  %cd /content/tf-models/research

  !python /content/tf-models/research/object_detection/inference_from_model.py  

## TF 2.0 drawback (Virtual env)
I know this is weird, a virtual machine in a Google Colab? Yes, unfortunately. This happens because the default Colab version of TF is 2.x, but the training must be done in version 1.x for compatibility reasons. I tried to adapt the TF 2.x to run the training but no sucess.

This will definelly change in a future version, but by now, this works!

Here we are creating a new venv and installing all the requirements, including TF 1.x.

**Note:** Change Colab's TF version to 1.x will not work once the inference script is adapted to run on version 2.x, the best to do is wait until a complete compatible version releases. You can also try to adapt the training script to work on 2.x, but I think you will spend some time on it.


In [ ]:
def create_venv():
  !virtualenv tf1
  !source tf1/bin/activate; pip install tensorflow==1.15
  !source tf1/bin/activate; pip install -q Cython contextlib2 pillow lxml matplotlib

  !source tf1/bin/activate; pip install -q pycocotools

  !source tf1/bin/activate; pip install gast==0.3.3

  !source tf1/bin/activate; pip install tf_slim
  !source tf1/bin/activate; pip install scipy
  %cd /content/tf-models/research
  !source tf1/bin/activate; protoc object_detection/protos/*.proto --python_out=.

  !source tf1/bin/activate; python object_detection/builders/model_builder_test.py
  !source tf1/bin/activate; pip install tensorflow-object-detection-api

# Iteration process
Here is where all the iteration is done, the methods we create before are called and do their job. The explanations are in code.

In [ ]:
import os
import sys
train_path = repo_dir_path + '/research/object_detection' # base directory

unlabel_data = len(os.listdir(train_path + '/unlabeled_data')) # number of remaining unlabeled images
train_count = 0

while train_count != 5: # my stop criterion is 5 iterations, you are free to change it!

  create_venv() # In the next iterations, this will run faster, as everything will be cached.
  
  train_model()
  print('MODEL TRAINED')

  if train_count > 0: # Delete the checkpoint folder after the first iteration, where it's created. Necessary to avoid errors
      fine_tune_model_path = repo_dir_path + '/research/fine_tuned_model'
      shutil.rmtree(fine_tune_model_path)

  fine_tune_model_name = 'fine_tuned_model' # define the checkpoint path for pipeline config 

  export_model() 
  print('MODEL EXPORTED') 

  inference_from_model() 
  print('INFERENCE DONE')

  generate_tfrecord(repo_dir_path) # new TF records!
  print('NEW TF RECORDs')

  unlabel_data = len(os.listdir(train_path + '/unlabeled_data'))
  print('REMAINING IMAGES', unlabel_data) # Check how many images are not labeled yet

  checkpoint_path = os.path.join('/content/tf-models/research', fine_tune_model_name)
  fine_tune_checkpoint = os.path.join(checkpoint_path, "model.ckpt")

  num_steps += 1500 if unlabel_data > 0 else 2000 # this is important, let's talk more about this below.

  change_pipeline() # new parameters in pipe config
  print('PIPELINE CHANGED')

  start_new_training_dir() 
  print('NEW TRAINING DIR')
  train_count += 1


A saída de streaming foi truncada nas últimas 5000 linhas.
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
INFO:tensorflow:Done calling model_fn.
I0721 01:07:50.111001 140136062637952 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-21T01:07:50Z
I0721 01:07:50.127454 140136062637952 evaluation.py:255] Starting evaluation at 2020-07-21T01:07:50Z
INFO:tensorflow:Graph was finalized.
I0721 01:07:50.617810 140136062637952 monitored_session.py:240] Graph was finalized.
2020-07-21 01:07:50.619097: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-21 01:07:50.619627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1

As you saw in my Medium Article, the epochs pattern will be a really important parameter. Dependending on your problem you may want to change this number to give more time to your model learn something. In this case, after no remaining unlabeled images, I increased the number of epochs by iteration, so the model could have more time to generalize the features. 

## Downloading the model!
If you want to deploy this model or test it "in field" you will need to download it into your machine.

In [ ]:
output_directory =  repo_dir_path + '/research/fine_tuned_model'

!ls {model_dir}
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

checkpoint_meta = os.path.join(os.path.abspath(output_directory), "model.ckpt.meta")
checkpoint_index = os.path.join(os.path.abspath(output_directory), "model.ckpt.index")
checkpoint_data = os.path.join(os.path.abspath(output_directory), "model.ckpt.data-00000-of-00001")
checkpoint = os.path.join(os.path.abspath(output_directory), "checkpoint")
saved_model = os.path.join(os.path.abspath(output_directory), "saved_model/saved_model.pb")

# uncomment this line to download logs too
#tf_log = os.path.join(os.path.abspath('/content/models/research/training/eval_0'), "events.out.tfevents.1576020673.0083b462c1a8")


In [ ]:
from google.colab import files
files.download(pb_fname)
files.download(label_map_pbtxt_fname)
files.download(pipeline_fname)
files.download(checkpoint_meta)
files.download(checkpoint_index)
files.download(checkpoint_data)
files.download(saved_model)
files.download(checkpoint)

## Check some automatically created labels!
You can download some images and XML files to check how your model is labeling your data. You may actually want to download the whole folder to get all your brand new labeled images!

In [ ]:
from google.colab import files
files.download('/content/tf-models/research/object_detection/train_images/dog.154.jpg') # my dog image and xml :)
files.download('/content/tf-models/research/object_detection/train_images/dog.xml.jpg')

# Some results
These are some images labeled automatically, please share this project if it helped you and contribute if you want :)


![texto alternativo](https://live.staticflickr.com/65535/50145046206_7231a8332c_b.jpg)
